# Dataframes

## Parameter tuning

In [117]:
import os
import re
import json
import pandas as pd
import numpy as np
from natsort import index_natsorted

### Make dataframes from each parameter setting run

In [118]:
# Change
run_name = '200521-105453/'

# Constant
project_path = os.path.dirname(os.path.abspath('.'))
directory_path = '/output/solstorm/'
run_path = project_path + directory_path + run_name
nbr_sims = 5
instance_key = 'instance'
best_obj_key = 'best obj'
avg_obj_key = 'obj'
max_time_key = 'max time'
min_time_key = 'min time'
avg_time_key = 'time'
max_iter_key = 'max iter'
min_iter_key = 'min iter'
avg_iter_key = 'iter'
parameter_key = 'parameters'
parameter_one_key = 'removal_lower_percentage'
parameter_two_key = 'removal_upper_percentage'

ordered_instance_li = ['T-9-9-1-1', 'T-11-13-1-1', 'T-13-16-2-1', 'T-15-17-2-1', 'T-17-21-3-1',
                       'T-19-22-2-1', 'T-21-27-3-1', 'T-23-27-3-1', 'T-25-31-4-1', 'T-27-34-5-1']

# instance to [best_obj, acc_obj, max_time, min_time, acc_time, max_iter, min_iter, acc_iter]
instance_to_data = {}
parameter_one_values = set()
parameter_two_values = set()
for file_name in os.listdir(run_path):
    split_name = re.split('_|\.', file_name)
    instance_name = split_name[0]
    is_history = split_name[2] == 'history'
    if is_history:
        with open(run_path + file_name) as file:
            history_json = json.load(file)
        
        parameter_one_value = history_json[parameter_key][parameter_one_key]
        parameter_two_value = history_json[parameter_key][parameter_two_key]
        parameter_one_values.add(parameter_one_value)
        parameter_two_values.add(parameter_two_value)
        
        obj = history_json['best_objective']
        time = history_json['runtime']
        it = history_json['number_of_iterations']

        if instance_name in instance_to_data:
            data = instance_to_data[instance_name]

            if obj < data[0]:
                data[0] = obj
            if time > data[2]:
                data[2] = time
            if time < data[3]:
                data[3] = time
            if it > data[5]:
                data[5] = it
            if it < data[6]:
                data[6] = it
            
            data[1] += obj
            data[4] += time
            data[7] += it
        else:
            instance_to_data[instance_name] = [obj, obj, time, time, time, it, it, it]
            
if len(parameter_one_values) > 1 or len(parameter_two_values) > 1:
    print('Multiple parameter values present in run directory!')
    
df = pd.DataFrame(columns=[instance_key, best_obj_key, avg_obj_key, max_time_key, min_time_key,
                           avg_time_key, max_iter_key, min_iter_key, avg_iter_key])

for instance in instance_to_data:
    data = instance_to_data[instance]
    best_objective = data[0]
    avg_objective = data[1] / nbr_sims
    max_time = data[2]
    min_time = data[3]
    avg_time = data[4] / nbr_sims
    max_iter = data[5]
    min_iter = data[6]
    avg_iter = data[7] / nbr_sims
    row = pd.Series({instance_key: instance, 
                     best_obj_key: best_objective,
                     avg_obj_key: avg_objective, 
                     max_time_key: max_time,
                     min_time_key: min_time,
                     avg_time_key: avg_time, 
                     max_iter_key: max_iter,
                     min_iter_key: min_iter,
                     avg_iter_key: avg_iter})
    df = df.append(row, ignore_index=True)

# df.sort_values(instance_key)

# Retrieve parameter values
for val in parameter_one_values:
    parameter_one_value = val
    break

for val in parameter_two_values:
    parameter_two_value = val
    break

file_name = f'dataframes/rp-{parameter_one_value}-{parameter_two_value}.pkl'

df.to_pickle(file_name)

### Functions

In [130]:
def sort_df(df, column_name):
    df = df.sort_values(by=column_name,
                        key=lambda x: np.argsort(index_natsorted(df[column_name])),
                        inplace=False)
    df = df.reset_index(drop=True)
    return df

def get_run_df(file_name, sort_column):
    run_df = pd.read_pickle(f'dataframes/{file_name}')
    run_df = sort_df(run_df, sort_column)
    return run_df

def get_sub_df(df, column_names, sort_column):
    sub = df[column_names].copy()
    sub = sort_df(sub, sort_column)
    return sub

def merge_sub_dfs(sub_dfs):
    df_total = pd.concat(sub_dfs, axis=1)

    # Drop duplicate instance columns
    li = [4, 8, 12, 16]
    df_total = df_total.iloc[:, [j for j, c in enumerate(df_total.columns) if j not in li]]
    
    df_total = df_total.round(1)
    return df_total

### Parameter: Removal interval

In [132]:
# The column to sort the rows by
sort_column = instance_key

df_rp_1 = get_run_df('rp-0.05-0.15.pkl', sort_column)
df_rp_2 = get_run_df('rp-0.05-0.3.pkl', sort_column)
df_rp_3 = get_run_df('rp-0.15-0.3.pkl', sort_column)
df_rp_4 = get_run_df('rp-0.15-0.5.pkl', sort_column)
df_rp_5 = get_run_df('rp-0.3-0.5.pkl', sort_column)

# The columns that should be selected from each sub df
columns = [instance_key, avg_obj_key, avg_time_key, avg_iter_key]

# Get all sub dfs (each representing )
one = get_sub_df(df_rp_1, columns, sort_column)
two = get_sub_df(df_rp_2, columns, sort_column)
three = get_sub_df(df_rp_3, columns, sort_column)
four = get_sub_df(df_rp_4, columns, sort_column)
five = get_sub_df(df_rp_5, columns, sort_column)

df_total = merge_sub_dfs([one, two, three, four, five])

df_total

,instance,obj,time,iter,obj,time,iter,obj,time,iter,obj,time,iter,obj,time,iter
0,T-9-9-1-1,3144.5,8.1,4999.0,3144.5,8.3,4999.0,3144.5,8.8,4999.0,3144.5,10.1,4999.0,3144.5,10.7,4999.0
1,T-11-13-1-1,7999.5,13.0,4999.0,7999.5,17.1,4999.0,7999.5,18.9,4999.0,7999.5,21.5,4999.0,7999.5,20.7,4999.0
2,T-13-16-2-1,3607.4,45.4,4999.0,3621.4,56.7,4999.0,3614.6,58.2,4999.0,3621.4,78.0,4999.0,3621.4,90.9,4999.0
3,T-15-17-2-1,4313.6,46.3,4999.0,4165.1,55.7,4999.0,4299.8,62.0,4999.0,4291.0,83.3,4999.0,4304.6,100.5,4999.0
4,T-17-21-3-1,4752.4,38.9,4999.0,4717.9,71.0,4999.0,4741.5,71.3,4999.0,4750.8,120.7,4999.0,4727.7,169.8,4999.0
5,T-19-22-2-1,8830.3,65.9,4999.0,8812.6,104.8,4999.0,8810.1,117.8,4999.0,8783.7,193.4,4999.0,8664.9,220.6,4999.0
6,T-21-27-3-1,5324.3,153.1,4999.0,5339.8,267.0,4999.0,5251.3,297.1,4999.0,5240.7,443.7,4999.0,5083.8,487.8,4999.0
7,T-23-27-3-1,5173.6,182.8,4999.0,4693.0,313.1,4999.0,4937.4,375.5,4999.0,4992.6,553.7,4999.0,4775.0,536.2,4880.2
8,T-25-31-4-1,6885.4,213.3,4999.0,6550.3,345.7,4999.0,6805.4,455.6,4999.0,6671.6,589.0,4917.4,6659.3,583.8,4469.6
9,T-27-34-5-1,7014.5,226.4,4999.0,7108.9,423.4,4999.0,6940.5,494.7,4999.0,6582.4,563.4,4899.0,6990.9,600.1,4048.6
